In [3]:
import psycopg
import numpy as np
import pandas as pd
from pgvector.psycopg import register_vector

table_name = 'vector_test'

# test_csv_path = "./data/similarity_vectors.csv"
# df = pd.read_csv(test_csv_path)
# df = df.to_numpy().flatten()
# print(df.shape)

#establishing the connection
with psycopg.connect("dbname=postgres user=billyslim password=''") as conn:
    register_vector(conn)
    # sim_query = """
    # SELECT id, embedding <-> (%s) AS distance
    # FROM vector_test 
    # ORDER BY distance ASC
    # LIMIT 5;
    # """

    
    # result = conn.execute(sim_query, (df,)).fetchall()
    
    # conn.execute('CREATE EXTENSION IF NOT EXISTS vector')
    # conn.execute(f'CREATE TABLE {table_name} (id bigserial PRIMARY KEY, embedding vector(10000))')
    index_query = f"CREATE INDEX ON {table_name} USING hnsw (embedding vector_l2_ops);"
    conn.execute(index_query)
    # conn.execute('DROP TABLE vector_index')
    # embedding = np.array([1, 2, 3])
    # conn.execute('INSERT INTO vector_test (embedding) VALUES (%s)', (embedding,))
    # result = conn.execute('SELECT embedding FROM vector_test ORDER BY embedding <-> %s LIMIT 5', (embedding,)).fetchall()
    # SELECT pg_size_pretty( pg_total_relation_size('vector_test')); size of table
    # print(result)

In [1]:
# https://github.com/pgvector/pgvector-python
# https://github.com/pgvector/pgvector

from pgvector_interface import PGvectorInterface
import numpy as np
import pandas as pd

csv_path = "./data/clustered_vectors.csv"
table_name = 'vector_test'
test_csv_path = "./data/similarity_vectors.csv"
embedding = pd.read_csv(test_csv_path)
embedding = embedding.to_numpy().flatten()
r = 0

pgvectordb = PGvectorInterface('postgres', 'billyslim')
# pgvectordb.create_table(table_name, 1000)
# pgvectordb.create_table(table_name, 1000, metrix="l2", index_types="hnsw")
# pgvectordb.drop_table(table_name)
# r = pgvectordb.get_size_of_table(table_name)
# embedding = np.array([1, 2, 3])
# pgvectordb.insert_single_vector(table_name, embedding)
# pgvectordb.insert_vector_from_csv(table_name, csv_path)
r = pgvectordb.get_size_of_table(table_name)
# for i in range(10000):
#     embedding = np.random.rand(1000)
#     r = pgvectordb.similarity_search(table_name, embedding, "l2")
# r = pgvectordb.similarity_search(table_name, embedding, "cosine")
# r=pgvectordb.get_rows_cnt(table_name)
pgvectordb.disconnect_server()
print(r)

[(19546112, 11272192, 8273920)]
19546112


In [ ]:
import chromadb

In [5]:
# Milvus
# pip install pymilvus milvus
# https://github.com/ytang07/milvus-projects/blob/main/text/embedding_comparison.ipynb
# https://milvus.io/docs/manage_databases.md
